In [8]:
my_file = open('./data/original/txts/the_dunwich_horror.txt')
raw_text = my_file.read()

In [14]:
from nltk.tokenize import TreebankWordTokenizer

source: http://www.nltk.org/book/ch01.html
https://www.nltk.org/book/ch03.html

In [15]:
tokenizer = TreebankWordTokenizer()

In [16]:
tokens = tokenizer.tokenize(raw_text)

In [17]:
from nltk.util import ngrams

In [20]:
list(ngrams(tokens,3))[:10]

[('The', 'Dunwich', 'Horror'),
 ('Dunwich', 'Horror', '―Gorgons'),
 ('Horror', '―Gorgons', ','),
 ('―Gorgons', ',', 'and'),
 (',', 'and', 'Hydras'),
 ('and', 'Hydras', ','),
 ('Hydras', ',', 'and'),
 (',', 'and', 'Chimaeras—dire'),
 ('and', 'Chimaeras—dire', 'stories'),
 ('Chimaeras—dire', 'stories', 'of')]

In [ ]:
# concordance: only works with nltk text